In [1]:
"""
Esse código é responsavel por tratar os dados, assim tratando os seguintes cenarios:
- Remover colunas que não serão utilizadas
- tratar linhas que possuem valores nulos
    - Colunas nulas numericas serão preenchidas com a -1
    - Colunas nulas String serão preenchidas com a string 'Desconhecido'
- tratar linhas que possuem valores duplicados
- tratar linhas que possuem valores fora do padrão
    - Colunas serão tratadas de acordo com o tipo de dado
    - float pode ser convertido para int
    - float 64 pode ser convertido para float 32 ou float 16
    - string pode ser convertida para categoria
    - int pode ser convertido para int8, int16, int32, int64
- Otimizar tipos de dados
- Salvar os tipos de dados otimizados em um arquivo externo
- Salvar os dados tratados em um novo arquivo
"""

"\nEsse código é responsavel por tratar os dados, assim tratando os seguintes cenarios:\n- Remover colunas que não serão utilizadas\n- tratar linhas que possuem valores nulos\n    - Colunas nulas numericas serão preenchidas com a -1\n    - Colunas nulas String serão preenchidas com a string 'Desconhecido'\n- tratar linhas que possuem valores duplicados\n- tratar linhas que possuem valores fora do padrão\n    - Colunas serão tratadas de acordo com o tipo de dado\n    - float pode ser convertido para int\n    - float 64 pode ser convertido para float 32 ou float 16\n    - string pode ser convertida para categoria\n    - int pode ser convertido para int8, int16, int32, int64\n- Otimizar tipos de dados\n- Salvar os tipos de dados otimizados em um arquivo externo\n- Salvar os dados tratados em um novo arquivo\n"

In [59]:
import seaborn as sns
import pandas as pd
import chardet
import json
import gc

In [3]:
input_file = '../microdados_enem_2023/DADOS/MICRODADOS_ENEM_2023.csv'
output_file = 'microdados_sem_nulos.csv'

# Verifica o encoding original do arquivo do arquivo (ISO-8859-1)
with open(input_file, 'rb') as f:
    resultado = chardet.detect(f.read(10000))
    encoding_original = resultado['encoding']

microdados_V1 = pd.read_csv(input_file, sep=';', encoding=encoding_original, low_memory=False)

In [4]:
#Remover coluna que não sera usada pois são numeros de inscrição aleatórios
microdados_V1.drop(columns=['NU_INSCRICAO'], inplace=True)
# microdados_V1.reset_index(drop=True, inplace=True)

In [5]:
#==================== DADOS NULOS ====================#

In [6]:
# Verifica a quantidade de linhas total do arquivo
total_linhas = microdados_V1.shape[0]
print(f'Total de linhas: {total_linhas}')

# Verifica a quantidade de linhas nulas
total_linhas_nulas = microdados_V1.isnull().sum().sum()
print(f'Total de linhas nulas: {total_linhas_nulas}')

Total de linhas: 3933955
Total de linhas nulas: 50024921


In [7]:
# Verificar informações sobre os dados
microdados_V1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933955 entries, 0 to 3933954
Data columns (total 75 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   NU_ANO                  int64  
 1   TP_FAIXA_ETARIA         int64  
 2   TP_SEXO                 object 
 3   TP_ESTADO_CIVIL         int64  
 4   TP_COR_RACA             int64  
 5   TP_NACIONALIDADE        int64  
 6   TP_ST_CONCLUSAO         int64  
 7   TP_ANO_CONCLUIU         int64  
 8   TP_ESCOLA               int64  
 9   TP_ENSINO               float64
 10  IN_TREINEIRO            int64  
 11  CO_MUNICIPIO_ESC        float64
 12  NO_MUNICIPIO_ESC        object 
 13  CO_UF_ESC               float64
 14  SG_UF_ESC               object 
 15  TP_DEPENDENCIA_ADM_ESC  float64
 16  TP_LOCALIZACAO_ESC      float64
 17  TP_SIT_FUNC_ESC         float64
 18  CO_MUNICIPIO_PROVA      int64  
 19  NO_MUNICIPIO_PROVA      object 
 20  CO_UF_PROVA             int64  
 21  SG_UF_PROVA             object 

In [8]:
# Localizando colunas nulas
microdados_V1.columns[microdados_V1.isnull().any()]

Index(['TP_ENSINO', 'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'CO_UF_ESC',
       'SG_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC',
       'TP_SIT_FUNC_ESC', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC',
       'CO_PROVA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',
       'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC',
       'TX_RESPOSTAS_MT', 'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC',
       'TX_GABARITO_MT', 'TP_STATUS_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2',
       'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO'],
      dtype='object')

In [9]:
"""
Selecionar colunas nulas relacionadas as notas do candidato
Verificar existencia de candidatos tiraram nota 0
Verificar existencia de valores igual a -1
"""
colunas_notas = [
    'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT',
    'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',
    'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4',
    'NU_NOTA_COMP5', 'NU_NOTA_REDACAO','TP_STATUS_REDACAO',
]

microdados_V1[colunas_notas].min() == 0

CO_PROVA_CN          False
CO_PROVA_CH          False
CO_PROVA_LC          False
CO_PROVA_MT          False
NU_NOTA_CN            True
NU_NOTA_CH            True
NU_NOTA_LC            True
NU_NOTA_MT            True
NU_NOTA_COMP1         True
NU_NOTA_COMP2         True
NU_NOTA_COMP3         True
NU_NOTA_COMP4         True
NU_NOTA_COMP5         True
NU_NOTA_REDACAO       True
TP_STATUS_REDACAO    False
dtype: bool

In [10]:
microdados_V1[colunas_notas].min() == -1

CO_PROVA_CN          False
CO_PROVA_CH          False
CO_PROVA_LC          False
CO_PROVA_MT          False
NU_NOTA_CN           False
NU_NOTA_CH           False
NU_NOTA_LC           False
NU_NOTA_MT           False
NU_NOTA_COMP1        False
NU_NOTA_COMP2        False
NU_NOTA_COMP3        False
NU_NOTA_COMP4        False
NU_NOTA_COMP5        False
NU_NOTA_REDACAO      False
TP_STATUS_REDACAO    False
dtype: bool

In [11]:
"""
Exitem candidatos que tiraram nota 0 na provas
Analisado que não existem valores -1 nas notas dos candidatos	
"""
microdados_V1[colunas_notas] = microdados_V1[colunas_notas].fillna(-1)
microdados_V1[colunas_notas].head()

,CO_PROVA_CN,CO_PROVA_CH,CO_PROVA_LC,CO_PROVA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,TP_STATUS_REDACAO
0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,1221.0,1193.0,1204.0,1211.0,502.0,498.9,475.6,363.2,140.0,200.0,100.0,120.0,140.0,700.0,1.0
3,1224.0,1192.0,1202.0,1214.0,459.0,508.5,507.2,466.7,140.0,200.0,160.0,180.0,200.0,880.0,1.0
4,1222.0,1191.0,1201.0,1212.0,402.5,379.2,446.9,338.3,120.0,120.0,120.0,120.0,80.0,560.0,1.0


In [12]:
"""
Tratar colunas de dtype string nulas
Sendo valores do tipo string, preencher com a string 'Desconhecido'
"""
colunas_gabarito = [
    'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC', 'TX_RESPOSTAS_MT',
    'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC', 'TX_GABARITO_MT',
]

microdados_V1[colunas_gabarito] = microdados_V1[colunas_gabarito].fillna('Desconhecido')
microdados_V1[colunas_gabarito].head()

,TX_RESPOSTAS_CN,TX_RESPOSTAS_CH,TX_RESPOSTAS_LC,TX_RESPOSTAS_MT,TX_GABARITO_CN,TX_GABARITO_CH,TX_GABARITO_LC,TX_GABARITO_MT
0,Desconhecido,Desconhecido,Desconhecido,Desconhecido,Desconhecido,Desconhecido,Desconhecido,Desconhecido
1,Desconhecido,Desconhecido,Desconhecido,Desconhecido,Desconhecido,Desconhecido,Desconhecido,Desconhecido
2,DBEBDCECCBCEBBBBDBABDDBBAABCBACDBACECCBAADEBB,ABDEADAADCDABDCADAEABCDDCBAADCCBEBCEBEBDBEAED,ACEBDCABAACAEBAECEBBBAAECBBDEADCAECCCEDDABEED,CEAEACCCDABCDAACEDDBAAEBABDDEEBDAECABDBCBCADE,DBEABDABDCACDBECDDDBCAAABBACCCADEBECCCEDAEEED,ACEEABAADCDAADEABCDABCDCABCBDADEBAECABADBCDAE,DBABBAEBAAAACDACDEDAACADBADBCCEACCCEAAECBBEBCA...,BCCDEEABCBEDCEABBEBDABDDADDADECAADDCCBEBEABCC
3,DEEBEACCCEBDDBDCCCAEEDCBAAADBCBEEEDCDAAECBEEC,DDAAEEBCCDEADBCDDCBAECABEBDEBDABECECEDCDDAEED,ADBDADAEEEACAABBACADCAEBBAAEBBCDEBBDDADDCADAA,EECBAEDEEDDDBBAADEECDBBBECEAACEAEECDBEDDBCDCB,CDDDABBABDBEABDECCEEEDCEDAEBABDCCAACCCADACDBE,DBAADEADCDCABABCDDEBAEABAECABAACECDAECBDAABCD,BBBDAABAEACCEEEDEACBCACAACAACAAAECBBEDBCCADBDE...,EBDADDAEBEACBEDCECCBEABCADEBCCBCCDEBDDAABBADD
4,AECCEAACDEABEEECDBAEEAAADDEABCBCEBACEEDCBEABD,CADEBCEDDEBCBAEBADDCECACADBDEBABDBDBEEDBBEADC,AABBACBCAEDABDADEDAACCAEEEECAACDCADBAEACDEAAE,CDBABEDCEEBBBDECDEBACCAABDEDCBECDECABBDBDEECC,CAAADCCCCDDDABDCACDBEEEDCEDAEECCDBEABDBABBAEB,CDAEECABAACEAADECBDAABCDCABADCDEABAABCDDEBADB,BBDABAAEBADACEEDCCDBADBDEDCCEBCACEACAACAACACBB...,DCECACCBDECBEEABEABDDAADDABBBCCBCCDDAEBDADEEB


In [13]:
""" 
Criação de uma nova colunas para identificar se o candidato esteve presente em todos os dias de prova
Extrair colunas relacionadas a presença do candidato e somar o valor de cada coluna
Onde será preenchido com 1 se o candidato esteve presente em todos os dias de prova e 0 caso contrário
Transformar o tipo de dado para boolean
"""
presenca_geral = ['TP_PRESENCA_CN',	'TP_PRESENCA_CH',	'TP_PRESENCA_LC', 'TP_PRESENCA_MT']

microdados_V1['TP_PRESENCA_GERAL'] = microdados_V1[presenca_geral].sum(axis=1).apply(
    lambda x: 1 if x>0 else 0
)
microdados_V1['TP_PRESENCA_GERAL'] = microdados_V1['TP_PRESENCA_GERAL'].astype('bool')

print(microdados_V1['TP_PRESENCA_GERAL'].loc[[71]])
print(microdados_V1['TP_PRESENCA_GERAL'].loc[[72]])
print(microdados_V1['TP_PRESENCA_GERAL'].loc[[73]])

71    True
Name: TP_PRESENCA_GERAL, dtype: bool
72    True
Name: TP_PRESENCA_GERAL, dtype: bool
73    False
Name: TP_PRESENCA_GERAL, dtype: bool


In [14]:
# Verfica a quantidade de linhas nulas restantes nas colunas tratadas
microdados_V1[colunas_gabarito].isna().sum().sum()
microdados_V1[colunas_notas].isna().sum().sum()

np.int64(0)

In [15]:
# Verifica a quantidade de colunas nulas restantes
colunas_com_nulos = microdados_V1.columns[microdados_V1.isnull().any()]

In [16]:
# Extraindo coluans com valores nulos do tipo number e aplicando valores -1
colunas_com_nulos_numericos = microdados_V1[colunas_com_nulos].select_dtypes(include='number').columns
microdados_V1[colunas_com_nulos_numericos] = microdados_V1[colunas_com_nulos_numericos].fillna(-1)
# Colunas que possuem valores nulos do tipo string, continuaram sem alteração
microdados_V1[colunas_com_nulos].head()

,TP_ENSINO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC
0,-1.0,-1.0,NaN,-1.0,NaN,-1.0,-1.0,-1.0
1,-1.0,-1.0,NaN,-1.0,NaN,-1.0,-1.0,-1.0
2,-1.0,-1.0,NaN,-1.0,NaN,-1.0,-1.0,-1.0
3,1.0,2304400.0,Fortaleza,23.0,CE,2.0,1.0,1.0
4,1.0,2311306.0,Quixadá,23.0,CE,2.0,1.0,1.0


In [17]:
# Verifica a quantidade de colunas nulas restantes
# Tratar colunas nulas do tipo string com a string 'Desconhecido'
colunas_com_nulos_string = microdados_V1.columns[microdados_V1.isnull().any()]
microdados_V1[colunas_com_nulos_string] = microdados_V1[colunas_com_nulos_string].fillna('Desconhecido')
print(microdados_V1[colunas_com_nulos_string].head())

  NO_MUNICIPIO_ESC     SG_UF_ESC
0     Desconhecido  Desconhecido
1     Desconhecido  Desconhecido
2     Desconhecido  Desconhecido
3        Fortaleza            CE
4          Quixadá            CE


In [18]:
microdados_V1.columns[microdados_V1.isnull().any()]

Index([], dtype='object')

In [19]:
#==================== OTIMIZAÇÂO DE DTYPES ====================#

In [20]:
microdados_V2 = microdados_V1.copy()
del microdados_V1
gc.collect()

0

In [21]:
microdados_V2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933955 entries, 0 to 3933954
Data columns (total 76 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   NU_ANO                  int64  
 1   TP_FAIXA_ETARIA         int64  
 2   TP_SEXO                 object 
 3   TP_ESTADO_CIVIL         int64  
 4   TP_COR_RACA             int64  
 5   TP_NACIONALIDADE        int64  
 6   TP_ST_CONCLUSAO         int64  
 7   TP_ANO_CONCLUIU         int64  
 8   TP_ESCOLA               int64  
 9   TP_ENSINO               float64
 10  IN_TREINEIRO            int64  
 11  CO_MUNICIPIO_ESC        float64
 12  NO_MUNICIPIO_ESC        object 
 13  CO_UF_ESC               float64
 14  SG_UF_ESC               object 
 15  TP_DEPENDENCIA_ADM_ESC  float64
 16  TP_LOCALIZACAO_ESC      float64
 17  TP_SIT_FUNC_ESC         float64
 18  CO_MUNICIPIO_PROVA      int64  
 19  NO_MUNICIPIO_PROVA      object 
 20  CO_UF_PROVA             int64  
 21  SG_UF_PROVA             object 

In [22]:
# Separar colunas por tipo de dado
print('Colunas do tipo int64:')
colunas_int64 = microdados_V2.select_dtypes(include='int64').columns
print(colunas_int64)

print('\nColunas do tipo float64:')
colunas_float64 = microdados_V2.select_dtypes(include='float64').columns
print(colunas_float64)

Colunas do tipo int64:
Index(['NU_ANO', 'TP_FAIXA_ETARIA', 'TP_ESTADO_CIVIL', 'TP_COR_RACA',
       'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA',
       'IN_TREINEIRO', 'CO_MUNICIPIO_PROVA', 'CO_UF_PROVA', 'TP_PRESENCA_CN',
       'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 'TP_LINGUA',
       'Q005'],
      dtype='object')

Colunas do tipo float64:
Index(['TP_ENSINO', 'CO_MUNICIPIO_ESC', 'CO_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC',
       'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC', 'CO_PROVA_CN', 'CO_PROVA_CH',
       'CO_PROVA_LC', 'CO_PROVA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC',
       'NU_NOTA_MT', 'TP_STATUS_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2',
       'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO'],
      dtype='object')


In [23]:
"""
Verificar se é possivel converter os valores do tipo float64 para int64
Verificar se todas as colunas do tipo float64 possuem valores inteiros
"""

resultados = {}
for coluna in colunas_float64:
    resultados[coluna] = microdados_V2[coluna].apply(lambda x: x.is_integer()).all()

for coluna, resultado in resultados.items():
    print(f'Colunas: {coluna} - Possui apenas valores inteiros: {resultado}')

Colunas: TP_ENSINO - Possui apenas valores inteiros: True
Colunas: CO_MUNICIPIO_ESC - Possui apenas valores inteiros: True
Colunas: CO_UF_ESC - Possui apenas valores inteiros: True
Colunas: TP_DEPENDENCIA_ADM_ESC - Possui apenas valores inteiros: True
Colunas: TP_LOCALIZACAO_ESC - Possui apenas valores inteiros: True
Colunas: TP_SIT_FUNC_ESC - Possui apenas valores inteiros: True
Colunas: CO_PROVA_CN - Possui apenas valores inteiros: True
Colunas: CO_PROVA_CH - Possui apenas valores inteiros: True
Colunas: CO_PROVA_LC - Possui apenas valores inteiros: True
Colunas: CO_PROVA_MT - Possui apenas valores inteiros: True
Colunas: NU_NOTA_CN - Possui apenas valores inteiros: False
Colunas: NU_NOTA_CH - Possui apenas valores inteiros: False
Colunas: NU_NOTA_LC - Possui apenas valores inteiros: False
Colunas: NU_NOTA_MT - Possui apenas valores inteiros: False
Colunas: TP_STATUS_REDACAO - Possui apenas valores inteiros: True
Colunas: NU_NOTA_COMP1 - Possui apenas valores inteiros: True
Colunas: 

In [24]:
"""
Selecionar colunas que possuem apenas valores float
Analizar valor maximo para verificar se é possivel converter para float16
Converter colunas float64 para float16
"""
colunas_float16 = [
    'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT'
]

print(f'Valor máximo encontrado: {microdados_V2[colunas_float16].max().max()}')
microdados_V2[colunas_float16] = microdados_V2[colunas_float16].astype('float16')
microdados_V2.info()

Valor máximo encontrado: 958.6
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933955 entries, 0 to 3933954
Data columns (total 76 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   NU_ANO                  int64  
 1   TP_FAIXA_ETARIA         int64  
 2   TP_SEXO                 object 
 3   TP_ESTADO_CIVIL         int64  
 4   TP_COR_RACA             int64  
 5   TP_NACIONALIDADE        int64  
 6   TP_ST_CONCLUSAO         int64  
 7   TP_ANO_CONCLUIU         int64  
 8   TP_ESCOLA               int64  
 9   TP_ENSINO               float64
 10  IN_TREINEIRO            int64  
 11  CO_MUNICIPIO_ESC        float64
 12  NO_MUNICIPIO_ESC        object 
 13  CO_UF_ESC               float64
 14  SG_UF_ESC               object 
 15  TP_DEPENDENCIA_ADM_ESC  float64
 16  TP_LOCALIZACAO_ESC      float64
 17  TP_SIT_FUNC_ESC         float64
 18  CO_MUNICIPIO_PROVA      int64  
 19  NO_MUNICIPIO_PROVA      object 
 20  CO_UF_PROVA             int64  
 21  

In [25]:
"""
Restante das colunas podem ser convertidas para int64 inicialmente e depois tratada conforme a necessidade
"""

colunas_float64=microdados_V2.select_dtypes(include='float64').columns
microdados_V2[colunas_float64] = microdados_V2[colunas_float64].astype('int64')
microdados_V2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933955 entries, 0 to 3933954
Data columns (total 76 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   NU_ANO                  int64  
 1   TP_FAIXA_ETARIA         int64  
 2   TP_SEXO                 object 
 3   TP_ESTADO_CIVIL         int64  
 4   TP_COR_RACA             int64  
 5   TP_NACIONALIDADE        int64  
 6   TP_ST_CONCLUSAO         int64  
 7   TP_ANO_CONCLUIU         int64  
 8   TP_ESCOLA               int64  
 9   TP_ENSINO               int64  
 10  IN_TREINEIRO            int64  
 11  CO_MUNICIPIO_ESC        int64  
 12  NO_MUNICIPIO_ESC        object 
 13  CO_UF_ESC               int64  
 14  SG_UF_ESC               object 
 15  TP_DEPENDENCIA_ADM_ESC  int64  
 16  TP_LOCALIZACAO_ESC      int64  
 17  TP_SIT_FUNC_ESC         int64  
 18  CO_MUNICIPIO_PROVA      int64  
 19  NO_MUNICIPIO_PROVA      object 
 20  CO_UF_PROVA             int64  
 21  SG_UF_PROVA             object 

In [26]:
"""
Converter colunas do tipo string seu respectivo tipo de acordo com seu tamanho 
"""
colunas_int64 = microdados_V2.select_dtypes(include='int64').columns

for col in colunas_int64:
  if microdados_V2[col].max() <= 127:
    microdados_V2[col] = microdados_V2[col].astype('int8')
  elif microdados_V2[col].max() <= 32767:
    microdados_V2[col] = microdados_V2[col].astype('int16')
  elif microdados_V2[col].max() <= 2147483647:
    microdados_V2[col] = microdados_V2[col].astype('int32')
  else:
    microdados_V2[col] = microdados_V2[col].astype('int64')

microdados_V2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933955 entries, 0 to 3933954
Data columns (total 76 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   NU_ANO                  int16  
 1   TP_FAIXA_ETARIA         int8   
 2   TP_SEXO                 object 
 3   TP_ESTADO_CIVIL         int8   
 4   TP_COR_RACA             int8   
 5   TP_NACIONALIDADE        int8   
 6   TP_ST_CONCLUSAO         int8   
 7   TP_ANO_CONCLUIU         int8   
 8   TP_ESCOLA               int8   
 9   TP_ENSINO               int8   
 10  IN_TREINEIRO            int8   
 11  CO_MUNICIPIO_ESC        int32  
 12  NO_MUNICIPIO_ESC        object 
 13  CO_UF_ESC               int8   
 14  SG_UF_ESC               object 
 15  TP_DEPENDENCIA_ADM_ESC  int8   
 16  TP_LOCALIZACAO_ESC      int8   
 17  TP_SIT_FUNC_ESC         int8   
 18  CO_MUNICIPIO_PROVA      int32  
 19  NO_MUNICIPIO_PROVA      object 
 20  CO_UF_PROVA             int8   
 21  SG_UF_PROVA             object 

In [27]:
# Extrair colunas do tipo object
colunas_object =  microdados_V2.select_dtypes(include=['object']).columns
print(colunas_object)

Index(['TP_SEXO', 'NO_MUNICIPIO_ESC', 'SG_UF_ESC', 'NO_MUNICIPIO_PROVA',
       'SG_UF_PROVA', 'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC',
       'TX_RESPOSTAS_MT', 'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC',
       'TX_GABARITO_MT', 'Q001', 'Q002', 'Q003', 'Q004', 'Q006', 'Q007',
       'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014', 'Q015', 'Q016',
       'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022', 'Q023', 'Q024', 'Q025'],
      dtype='object')


In [ ]:
# for col in colunas_object:
#   print(f'{col}: {microdados_V2[col].unique()} valores únicos')

In [38]:
# Analisar quais colunas tem a possibilidade de serem convertidas para categoria

resultados = {}

for col in colunas_object:
    # Verificar se a quantidade de valores unicos é menor que 50% do total de linhas
    if microdados_V2[col].nunique()/microdados_V2[col].shape[0] < 0.01:
        resultados[col] = True
    else:
        resultados[col] = False

for col, resultado in resultados.items():
    print(f'Coluna: {col} - Pode ser convertida para categoria: {resultado}')



Coluna: TP_SEXO - Pode ser convertida para categoria: True
Coluna: NO_MUNICIPIO_ESC - Pode ser convertida para categoria: True
Coluna: SG_UF_ESC - Pode ser convertida para categoria: True
Coluna: NO_MUNICIPIO_PROVA - Pode ser convertida para categoria: True
Coluna: SG_UF_PROVA - Pode ser convertida para categoria: True
Coluna: TX_RESPOSTAS_CN - Pode ser convertida para categoria: False
Coluna: TX_RESPOSTAS_CH - Pode ser convertida para categoria: False
Coluna: TX_RESPOSTAS_LC - Pode ser convertida para categoria: False
Coluna: TX_RESPOSTAS_MT - Pode ser convertida para categoria: False
Coluna: TX_GABARITO_CN - Pode ser convertida para categoria: True
Coluna: TX_GABARITO_CH - Pode ser convertida para categoria: True
Coluna: TX_GABARITO_LC - Pode ser convertida para categoria: True
Coluna: TX_GABARITO_MT - Pode ser convertida para categoria: True
Coluna: Q001 - Pode ser convertida para categoria: True
Coluna: Q002 - Pode ser convertida para categoria: True
Coluna: Q003 - Pode ser convert

In [45]:
# Converter colunas selecionadas para categoria
for col in colunas_object:
    if resultados[col]:
        microdados_V2[col] = microdados_V2[col].astype('category')

microdados_V2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933955 entries, 0 to 3933954
Data columns (total 76 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   NU_ANO                  int16   
 1   TP_FAIXA_ETARIA         int8    
 2   TP_SEXO                 category
 3   TP_ESTADO_CIVIL         int8    
 4   TP_COR_RACA             int8    
 5   TP_NACIONALIDADE        int8    
 6   TP_ST_CONCLUSAO         int8    
 7   TP_ANO_CONCLUIU         int8    
 8   TP_ESCOLA               int8    
 9   TP_ENSINO               int8    
 10  IN_TREINEIRO            bool    
 11  CO_MUNICIPIO_ESC        int32   
 12  NO_MUNICIPIO_ESC        category
 13  CO_UF_ESC               int8    
 14  SG_UF_ESC               category
 15  TP_DEPENDENCIA_ADM_ESC  int8    
 16  TP_LOCALIZACAO_ESC      int8    
 17  TP_SIT_FUNC_ESC         int8    
 18  CO_MUNICIPIO_PROVA      int32   
 19  NO_MUNICIPIO_PROVA      category
 20  CO_UF_PROVA             int8    
 21  SG_UF_PR

In [30]:
# Converter colunas restantes do tipo object para string
colunas_object = microdados_V2.select_dtypes(include=['object']).columns
microdados_V2[colunas_object] = microdados_V2[colunas_object].astype('string')
microdados_V2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933955 entries, 0 to 3933954
Data columns (total 76 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   NU_ANO                  int16  
 1   TP_FAIXA_ETARIA         int8   
 2   TP_SEXO                 string 
 3   TP_ESTADO_CIVIL         int8   
 4   TP_COR_RACA             int8   
 5   TP_NACIONALIDADE        int8   
 6   TP_ST_CONCLUSAO         int8   
 7   TP_ANO_CONCLUIU         int8   
 8   TP_ESCOLA               int8   
 9   TP_ENSINO               int8   
 10  IN_TREINEIRO            int8   
 11  CO_MUNICIPIO_ESC        int32  
 12  NO_MUNICIPIO_ESC        string 
 13  CO_UF_ESC               int8   
 14  SG_UF_ESC               string 
 15  TP_DEPENDENCIA_ADM_ESC  int8   
 16  TP_LOCALIZACAO_ESC      int8   
 17  TP_SIT_FUNC_ESC         int8   
 18  CO_MUNICIPIO_PROVA      int32  
 19  NO_MUNICIPIO_PROVA      string 
 20  CO_UF_PROVA             int8   
 21  SG_UF_PROVA             string 

In [46]:
# Mapear colunas 'TP_SEXO' para valores booleanos
microdados_V2['TP_SEXO'] = microdados_V2['TP_SEXO'].map({'M': 1, 'F': 0})

In [47]:
# Extrair colunas que podem ser convertidas para o tipo boolean
colunas_bool = []

for col in microdados_V2.columns:
    if set(microdados_V2[col].unique()) == {0,1}:
        colunas_bool.append(col)

for col in colunas_bool:
    print(f'Coluna: {col} - é booleana')

Coluna: IN_TREINEIRO - é booleana
Coluna: TP_LINGUA - é booleana
Coluna: TP_PRESENCA_GERAL - é booleana


In [48]:
# Converter colunas para o tipo boolean
microdados_V2[colunas_bool] = microdados_V2[colunas_bool].astype('bool')
microdados_V2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933955 entries, 0 to 3933954
Data columns (total 76 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   NU_ANO                  int16   
 1   TP_FAIXA_ETARIA         int8    
 2   TP_SEXO                 float64 
 3   TP_ESTADO_CIVIL         int8    
 4   TP_COR_RACA             int8    
 5   TP_NACIONALIDADE        int8    
 6   TP_ST_CONCLUSAO         int8    
 7   TP_ANO_CONCLUIU         int8    
 8   TP_ESCOLA               int8    
 9   TP_ENSINO               int8    
 10  IN_TREINEIRO            bool    
 11  CO_MUNICIPIO_ESC        int32   
 12  NO_MUNICIPIO_ESC        category
 13  CO_UF_ESC               int8    
 14  SG_UF_ESC               category
 15  TP_DEPENDENCIA_ADM_ESC  int8    
 16  TP_LOCALIZACAO_ESC      int8    
 17  TP_SIT_FUNC_ESC         int8    
 18  CO_MUNICIPIO_PROVA      int32   
 19  NO_MUNICIPIO_PROVA      category
 20  CO_UF_PROVA             int8    
 21  SG_UF_PR

In [49]:

# Mapear os tipos de dados otimizados e salvar um arquivo externo do tipo json para ser utilizado posteriormente
dtypes_dict = {col: str(microdados_V2[col].dtype) for col in microdados_V2.columns}
with open('dtypes.json', 'w') as f:
    json.dump(dtypes_dict, f)

In [ ]:
#==================== TRATAMENTO DE OUTLIERS ====================#

In [51]:
microdados_V3 = microdados_V2.copy()
del microdados_V2
gc.collect()

7

In [52]:
# Seleciona colunas numéricas e verifica colunas que possuem outliers
colunas_numerico = microdados_V3.select_dtypes(include=['number']).columns
colunas_outlier = []

for col in colunas_numerico:
    Q1 = microdados_V3[col].quantile(0.25)
    Q3 = microdados_V3[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = microdados_V3[(microdados_V3[col] < lower_bound) | (microdados_V3[col] > upper_bound)]
    if not outliers.empty:
        colunas_outlier.append(col)

print("Colunas com outliers:", colunas_outlier)

Colunas com outliers: ['TP_FAIXA_ETARIA', 'TP_ESTADO_CIVIL', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'CO_MUNICIPIO_ESC', 'CO_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC', 'CO_MUNICIPIO_PROVA', 'CO_UF_PROVA', 'TP_STATUS_REDACAO', 'Q005']


In [ ]:
# Analisado que as colunas outlires são referentes a colunas categoricas então não é necessário tratar outliers
# Nenhuma colunas referente a notas possui outliers

#==================== SALVAR DADOS TRATADOS ====================#

In [64]:
with open('microdados_tratado.csv', 'w') as f:
    microdados_V3.to_csv(f, index=False)